In [1]:
import mysql.connector
import requests



In [2]:
conn_relacional = mysql.connector.connect(
    host="127.0.0.1",
    port=3306,
    user="root",
    password="0101",
    database="relacional",
)

conn_dimensional = mysql.connector.connect(
    host="127.0.0.1",
    port=3306,
    user="root",
    password="0101",
    database="dimensional",
)

cursor_relacional = conn_relacional.cursor()
cursor_dimensional = conn_dimensional.cursor()

In [3]:
def pais_para_continente(pais):
    url = f"https://restcountries.com/v3.1/translation/{pais.lower()}"
    response = requests.get(url)

    if response.status_code != 200:
        return None
    
    data = response.json()[0]
    continente = data['region']

    traducao = {
        'Africa': 'África',
        'Americas': 'América',
        'Europe': 'Europa',
        'Asia': 'Ásia',
        'Oceania': 'Oceania',
        'Antarctic': 'Antártida'
    }
    return traducao.get(continente, continente)

## Localidade

In [4]:
cursor_relacional.execute(
    "SELECT idPais, nmPais FROM pais"
)
paises = cursor_relacional.fetchall()

cursor_relacional.execute(
    "SELECT idEstado, nmEstado FROM estado"
)
estados = cursor_relacional.fetchall()

cursor_relacional.execute(
    "SELECT idCidade, nmCidade FROM cidade"
)
cidades = cursor_relacional.fetchall()

continentes = []
for id_pais, nome_pais in paises:
    continentes.append(pais_para_continente(nome_pais))
    
# Inserts
cursor_dimensional.executemany(
    "INSERT INTO d_localidade (idLocalidade, pais, continente) VALUES (%s, %s, %s)",
    [(i+1, paises[0][1], continentes[0]) for i in range(len(estados))],
)

## Usuário

In [5]:
from datetime import datetime

def get_years(date, actual_date):
    return actual_date.year - date.year + 1

In [6]:
from datetime import datetime

cursor_relacional.execute(
    "SELECT idUsuario, nome, dtNascimento FROM usuario",
)
usuarios = cursor_relacional.fetchall()

actual_date = datetime.now().date()
dados = []
for id_usuario, nome, dt_nascimento in usuarios:
    faixa_etaria = get_years(dt_nascimento, actual_date) // 10
    dados.append((id_usuario, nome, faixa_etaria))

cursor_dimensional.executemany(
    "INSERT INTO d_usuario (idUsuario, nome, faixa_etaria) VALUES (%s, %s, %s)",
    dados,
)

## Pagamento

In [7]:
# campos: idPagamento, vlrRecebido
cursor_relacional.execute(
    "SELECT idPagamento, vlrRecebido FROM pagamento",
)
pagamentos = cursor_relacional.fetchall()

cursor_dimensional.executemany(
    "INSERT INTO d_Pagamento (idPagamento, vlrRecebido) VALUES (%s,%s)",
    pagamentos,
)

## Plano

In [8]:
cursor_relacional.execute(
    "SELECT idPlano, nmPlano FROM plano",
)
planos = cursor_relacional.fetchall()

cursor_dimensional.executemany(
    "INSERT INTO d_plano (idPlano, nmPlano) VALUES (%s, %s)",
    planos
)

## Assinatura

In [9]:
cursor_relacional.execute(
    "SELECT idAssinatura, Usuario_idUsuario, dtInicio FROM assinatura"
)
assinaturas = cursor_relacional.fetchall()

for id_assinatura, id_usuario, dt_inicio in assinaturas:
    # TODO: Tentar refazer usando dados já inseridos no banco dimensional
    cursor_relacional.execute(
        "SELECT pais_idPais, cidade_idCidade, estado_idEstado FROM usuario WHERE idUsuario=%s",
        (id_usuario,)
    )
    usuario = cursor_relacional.fetchone()
    pais_id, cidade_id, estado_id = usuario

    cursor_relacional.execute(
        "SELECT nmPais FROM pais WHERE idPais=%s",
        (pais_id,)
    )
    pais = cursor_relacional.fetchone()[0]

    cursor_relacional.execute(
        "SELECT nmEstado FROM estado WHERE idEstado=%s",
        (estado_id,)
    )
    estado = cursor_relacional.fetchone()[0]

    cursor_relacional.execute(
        "SELECT nmCidade FROM cidade WHERE idCidade=%s",
        (cidade_id,)
    )
    cidade = cursor_relacional.fetchone()[0]

    cursor_dimensional.execute(
        """
        SELECT idLocalidade FROM d_localidade
        WHERE pais=%s AND estado=%s AND cidade=%s
        """,
        (pais, estado, cidade)
    )
    localidade = cursor_dimensional.fetchone()[0]

    cursor_dimensional.execute(
        """
        SELECT idData FROM d_data
        WHERE mes=%s AND ano=%s
        """,
        (dt_inicio.month, dt_inicio.year)
    )
    data = cursor_dimensional.fetchall()
    print(dt_inicio.month, dt_inicio.year, len(data))

ProgrammingError: 1054 (42S22): Unknown column 'estado' in 'where clause'

## Receita

In [ ]:
cursor_relacional.execute(
    """SELECT idPagamento, vlrRecebido, Assinatura_idAssinatura, dtPagamento FROM pagamento""",
)
pagamentos = cursor_relacional.fetchall()

dados_receita = []
for id_pagamento, valor_recebido, id_assinatura, dt_pagamento in pagamentos:
    
    print(id_pagamento, valor_recebido, id_assinatura, dt_pagamento)
    
    # Assinatura
    cursor_relacional.execute(
        """SELECT Plano_idPlano FROM assinatura WHERE idAssinatura=%s""",
        (id_assinatura,)
    )
    plano_id = cursor_relacional.fetchone()[0]
    
    # idPlano
    cursor_dimensional.execute(
        """SELECT idPlano FROM d_plano WHERE idPlano=%s""",
        (plano_id,)
    )
    d_plano_id = cursor_dimensional.fetchone()[0]
    
    # idPagamento
    cursor_dimensional.execute(
        """SELECT idPagamento FROM d_pagamento WHERE idPagamento=%s""",
        (id_pagamento,)
    )
    d_pagamento_id = cursor_dimensional.fetchone()[0]
    
    # idData
    cursor_dimensional.execute(
        """
        SELECT idData FROM d_data
        WHERE mes=%s AND ano=%s
        """,
        (dt_pagamento.month, dt_pagamento.year)
    )
    d_data_id = cursor_dimensional.fetchall()
    print(dt_pagamento.month, dt_pagamento.year, len(d_data_id))
    
    dados_receita.append(
        (id_pagamento, valor_recebido, d_plano_id, d_pagamento_id, d_data_id)
    )

    

1 15.9 1 2019-8-7


AttributeError: 'str' object has no attribute 'month'

In [ ]:
# conn_dimensional.commit()
